In [1]:
import os
import pandas as pd
import csv
import pickle as pkl
import numpy as np
from fasttext import FastText
import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch import nn
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import datetime
from nltk import pos_tag
from tqdm import tqdm

In [2]:
all_user_va_tags = pkl.load(open('./../part-1/all_user_va_words.pkl', 'rb'))

In [3]:
all_user_va_songs = pkl.load(open('./../part-2/all_user_va_songs.pkl', 'rb'))

In [4]:
all_user_va_songs

{'48e9a7f191695247670d622319fba921.csv': array([0.40087472, 0.64677849]),
 '1bc8474dc465d16f5c2a424b2a7427e7.csv': array([0.48954701, 0.79345136]),
 '2bdf82895c47ce2fd668fd6369da78db.csv': array([0.41477034, 0.52946535]),
 'cebe7bb75898cb69fe9bb992d10cf954.csv': array([0.48669383, 0.68128424]),
 '929e1073f2ec54bf04e18ca8f33a1724.csv': array([0.44303954, 0.60595444]),
 'e81cc3349b02197a403e412ad57157cc.csv': array([0.36866179, 0.55611245]),
 '3ae99483f8e0b14ff7c264625c96c507.csv': array([0.48262414, 0.63708753]),
 'd125a06735486fa936de1f856e01bbb4.csv': array([0.44504576, 0.85691741]),
 '003534c470fd624743ee3f7acef28a53.csv': array([0.44783991, 0.83974478]),
 '05ed17d0ecc78208ba5d6a3661a9fc95.csv': array([0.50977877, 0.71952558]),
 '20f3da7086adafce8a7adf3a85c67c7a.csv': array([0.42233682, 0.61051024]),
 'ae37da73621a312704f806a04664d5ae.csv': array([0.45652895, 0.54574053]),
 'ff7aefe14748d4cfed2a7f6cbc6cb226.csv': array([0.4372958 , 0.68057808]),
 '2fe1a2176412d3a3307a28047d16dfbc.csv

In [5]:
all_user_va_tags

{'48e9a7f191695247670d622319fba921.csv': array([0.62776437, 0.44471566]),
 '1bc8474dc465d16f5c2a424b2a7427e7.csv': array([0.60038346, 0.44718212]),
 '2bdf82895c47ce2fd668fd6369da78db.csv': array([0.6284668 , 0.43972123]),
 'cebe7bb75898cb69fe9bb992d10cf954.csv': array([0.6467357 , 0.44958853]),
 '929e1073f2ec54bf04e18ca8f33a1724.csv': array([0.63511963, 0.44009491]),
 'e81cc3349b02197a403e412ad57157cc.csv': array([0.62639084, 0.43650207]),
 '3ae99483f8e0b14ff7c264625c96c507.csv': array([0.64190446, 0.44145089]),
 'd125a06735486fa936de1f856e01bbb4.csv': array([0.6496025, 0.4453403]),
 '003534c470fd624743ee3f7acef28a53.csv': array([0.61486697, 0.43366431]),
 '05ed17d0ecc78208ba5d6a3661a9fc95.csv': array([0.61232984, 0.43005474]),
 '20f3da7086adafce8a7adf3a85c67c7a.csv': array([0.62350067, 0.43497552]),
 'ae37da73621a312704f806a04664d5ae.csv': array([0.63573935, 0.43915293]),
 'ff7aefe14748d4cfed2a7f6cbc6cb226.csv': array([0.62891423, 0.43685042]),
 '2fe1a2176412d3a3307a28047d16dfbc.csv':

In [6]:
final_X = dict()
for i in all_user_va_songs.keys():
    final_X[i] = torch.from_numpy(np.append(all_user_va_songs[i], all_user_va_tags[i]))

In [7]:
final_X

{'48e9a7f191695247670d622319fba921.csv': tensor([0.4009, 0.6468, 0.6278, 0.4447], dtype=torch.float64),
 '1bc8474dc465d16f5c2a424b2a7427e7.csv': tensor([0.4895, 0.7935, 0.6004, 0.4472], dtype=torch.float64),
 '2bdf82895c47ce2fd668fd6369da78db.csv': tensor([0.4148, 0.5295, 0.6285, 0.4397], dtype=torch.float64),
 'cebe7bb75898cb69fe9bb992d10cf954.csv': tensor([0.4867, 0.6813, 0.6467, 0.4496], dtype=torch.float64),
 '929e1073f2ec54bf04e18ca8f33a1724.csv': tensor([0.4430, 0.6060, 0.6351, 0.4401], dtype=torch.float64),
 'e81cc3349b02197a403e412ad57157cc.csv': tensor([0.3687, 0.5561, 0.6264, 0.4365], dtype=torch.float64),
 '3ae99483f8e0b14ff7c264625c96c507.csv': tensor([0.4826, 0.6371, 0.6419, 0.4415], dtype=torch.float64),
 'd125a06735486fa936de1f856e01bbb4.csv': tensor([0.4450, 0.8569, 0.6496, 0.4453], dtype=torch.float64),
 '003534c470fd624743ee3f7acef28a53.csv': tensor([0.4478, 0.8397, 0.6149, 0.4337], dtype=torch.float64),
 '05ed17d0ecc78208ba5d6a3661a9fc95.csv': tensor([0.5098, 0.7195,

In [8]:
df = pd.read_csv('./../score/scores.csv')
df

,userID,unhealthy,k10
0,a21ee741f8dfa332b639e4f8ed286910.csv,13,14
1,cb73e271b112842db497dcf730827d2e.csv,13,25
2,a384c08f1ae51c6592dfebef6f4f8933.csv,9,31
3,3c09ebec28e30929dfb69c3659c15068.csv,16,18
4,4ffb277278816d11b1efc8cf12336d8f.csv,12,17
...,...,...,...
536,18fe71adbba985232eadc61dacb14887.csv,11,13
537,a425fef722bab4faf3e2d840112b1a21.csv,15,33
538,4803d6fc29b815ccca8d1d44e6e357fa.csv,27,25
539,120098859dcc2b3d864e57be165a76cd.csv,10,12


In [9]:
user_ids = df['userID']
unhealthy = df['unhealthy']
k10 = df['k10']

In [10]:
final_Y = dict()
for i in range(len(user_ids)):
    final_Y[user_ids[i]] = torch.tensor([[unhealthy[i], k10[i], unhealthy[i]*k10[i]/10]], dtype=torch.float64)

In [11]:
final_Y

{'a21ee741f8dfa332b639e4f8ed286910.csv': tensor([[13.0000, 14.0000, 18.2000]], dtype=torch.float64),
 'cb73e271b112842db497dcf730827d2e.csv': tensor([[13.0000, 25.0000, 32.5000]], dtype=torch.float64),
 'a384c08f1ae51c6592dfebef6f4f8933.csv': tensor([[ 9.0000, 31.0000, 27.9000]], dtype=torch.float64),
 '3c09ebec28e30929dfb69c3659c15068.csv': tensor([[16.0000, 18.0000, 28.8000]], dtype=torch.float64),
 '4ffb277278816d11b1efc8cf12336d8f.csv': tensor([[12.0000, 17.0000, 20.4000]], dtype=torch.float64),
 '4e9250bd2c527712ea8123137140d40b.csv': tensor([[11.0000, 18.0000, 19.8000]], dtype=torch.float64),
 'e9ad4919dead1b391edfec4398bb4bd5.csv': tensor([[13.0000, 33.0000, 42.9000]], dtype=torch.float64),
 '32f9391bec65796d5ae869df0084956a.csv': tensor([[ 8.0000, 12.0000,  9.6000]], dtype=torch.float64),
 'c6f5a256200b9582caa459b1053816d7.csv': tensor([[11., 20., 22.]], dtype=torch.float64),
 '941f769992f52ebe04adb36784087a7a.csv': tensor([[12.0000, 17.0000, 20.4000]], dtype=torch.float64),
 '

In [12]:
users = final_X.keys()
X = []
Y = []
for i in users:
    X.append(final_X[i])
    Y.append(final_Y[i])

In [13]:
X

[tensor([0.4009, 0.6468, 0.6278, 0.4447], dtype=torch.float64),
 tensor([0.4895, 0.7935, 0.6004, 0.4472], dtype=torch.float64),
 tensor([0.4148, 0.5295, 0.6285, 0.4397], dtype=torch.float64),
 tensor([0.4867, 0.6813, 0.6467, 0.4496], dtype=torch.float64),
 tensor([0.4430, 0.6060, 0.6351, 0.4401], dtype=torch.float64),
 tensor([0.3687, 0.5561, 0.6264, 0.4365], dtype=torch.float64),
 tensor([0.4826, 0.6371, 0.6419, 0.4415], dtype=torch.float64),
 tensor([0.4450, 0.8569, 0.6496, 0.4453], dtype=torch.float64),
 tensor([0.4478, 0.8397, 0.6149, 0.4337], dtype=torch.float64),
 tensor([0.5098, 0.7195, 0.6123, 0.4301], dtype=torch.float64),
 tensor([0.4223, 0.6105, 0.6235, 0.4350], dtype=torch.float64),
 tensor([0.4565, 0.5457, 0.6357, 0.4392], dtype=torch.float64),
 tensor([0.4373, 0.6806, 0.6289, 0.4369], dtype=torch.float64),
 tensor([0.4439, 0.7053, 0.6364, 0.4421], dtype=torch.float64),
 tensor([0.4799, 0.6208, 0.6199, 0.4345], dtype=torch.float64),
 tensor([0.3620, 0.5917, 0.6250, 0.4365]

In [14]:
Y

[tensor([[28.0000, 28.0000, 78.4000]], dtype=torch.float64),
 tensor([[13.0000, 14.0000, 18.2000]], dtype=torch.float64),
 tensor([[17.0000, 32.0000, 54.4000]], dtype=torch.float64),
 tensor([[17., 30., 51.]], dtype=torch.float64),
 tensor([[ 9.0000, 21.0000, 18.9000]], dtype=torch.float64),
 tensor([[17.0000, 36.0000, 61.2000]], dtype=torch.float64),
 tensor([[14.0000, 12.0000, 16.8000]], dtype=torch.float64),
 tensor([[14., 15., 21.]], dtype=torch.float64),
 tensor([[13.0000, 26.0000, 33.8000]], dtype=torch.float64),
 tensor([[11.0000, 24.0000, 26.4000]], dtype=torch.float64),
 tensor([[ 9.0000, 24.0000, 21.6000]], dtype=torch.float64),
 tensor([[11.0000, 14.0000, 15.4000]], dtype=torch.float64),
 tensor([[13.0000, 28.0000, 36.4000]], dtype=torch.float64),
 tensor([[13.0000, 23.0000, 29.9000]], dtype=torch.float64),
 tensor([[ 29.0000,  37.0000, 107.3000]], dtype=torch.float64),
 tensor([[24.0000, 33.0000, 79.2000]], dtype=torch.float64),
 tensor([[17., 20., 34.]], dtype=torch.float6

In [15]:
pkl.dump(X, open('X.pkl', 'wb'))
pkl.dump(Y, open('Y.pkl', 'wb'))